## import Libraries

In [10]:
import numpy as np
import os 
import cv2 as cv
import tensorflow as tf


## generate training data and test data

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_dir = 'train'
val_dir = 'test'
train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

train_generator = train_gen.flow_from_directory( train_dir,
                                                target_size=(48,48),
                                                batch_size=64,
                                                color_mode="grayscale",
                                                class_mode='categorical')

test_generator = test_gen.flow_from_directory( val_dir,
                                                target_size=(48,48),
                                                batch_size=64,
                                                color_mode="grayscale",
                                                class_mode='categorical')


Found 28710 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D

In [13]:
tf.keras.backend.clear_session()
input_shape = (48,48,1)
emotion_model = Sequential()
emotion_model.add(Conv2D(4,3,activation='relu', padding = 'same', input_shape=input_shape))
emotion_model.add(Dropout(0.1))
emotion_model.add(MaxPooling2D((2,2))) # output size = 24x24x3
emotion_model.add(Conv2D(8,3,activation='relu', padding = 'same'))
emotion_model.add(Dropout(0.1))
emotion_model.add(MaxPooling2D((3,3))) # output size = 8x8x9
emotion_model.add(Conv2D(24,3,activation='relu', padding = 'same'))
emotion_model.add(Dropout(0.1))
emotion_model.add(MaxPooling2D((3,3))) # output size = 4x4x18
emotion_model.add(Conv2D(48,3,activation='relu', padding = 'same'))
emotion_model.add(Dropout(0.1))
emotion_model.add(MaxPooling2D((2,2))) # output size = 1x1x36
emotion_model.add(Flatten())
emotion_model.add(Dense(7,activation='softmax'))

In [14]:
emotion_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 4)         40        
_________________________________________________________________
dropout (Dropout)            (None, 48, 48, 4)         0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 8)         296       
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 8)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 8)           0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 24)          1

In [ ]:
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=test_generator,

        validation_steps=7178 // 64)
emotion_model.save_weights('model.h5')


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
291/448 [==================>...........] - ETA: 2:57 - loss: 1.8557 - accuracy: 0.2426

In [15]:
emotion_model.load_weights('model.h5')

In [6]:
import cv2
cv2.ocl.setUseOpenCL(False)
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    bounding_box = cv2.CascadeClassifier('/home/shivam/.local/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Video', cv2.resize(frame,(1200,860),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

error: OpenCV(4.1.2) /io/opencv/modules/objdetect/src/cascadedetect.cpp:1689: error: (-215:Assertion failed) !empty() in function 'detectMultiScale'


In [7]:
cap.release()
cv2.destroyAllWindows()